In [ ]:
import numpy as np
import cv2

In [ ]:
def change_brightness(img, alpha, beta):
    img_new = np.asarray(alpha*img + beta, dtype=int)   # cast pixel values to int
    img_new[img_new>255] = 255
    img_new[img_new<0] = 0
    return np.array(img_new, dtype=np.uint8)

In [ ]:
image_eye_left_1 = cv2.imread("D:/User/ESP32_DOIT/scripts/kitchenrobot.jpg")
image_eye_left_2 = cv2.imread("D:/User/ESP32_DOIT/scripts/smilerobot.jpg")
image_eye_left_1 = cv2.resize(image_eye_left_1, [523,630])
image_eye_left_2 = cv2.resize(image_eye_left_2, [523,630])

print(image_eye_left_1.shape)
print(image_eye_left_2.shape)
# image_processed = change_brightness(image, 1, 35)
eyes = cv2.hconcat([image_eye_left_1, image_eye_left_2])

cv2.imshow("image", eyes)
cv2.imwrite("imagerobot.png", eyes)
cv2.waitKey(0)

In [ ]:
import os
import pandas as pd

path = 'D:/User/ESP32_DOIT/scripts/image_102.txt'
# read text file into pandas DataFrame
df = pd.read_csv(path, sep=" ", header=None)

# display DataFrame
print(df[0])
for i in range(len(df[0])):
    df[0][i] = 45
    

print(df[0])
df.to_csv(path, sep=' ', index=False, header=False)


In [ ]:
names = [ 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
         'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
         'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
         'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
         'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
         'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
         'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
         'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
         'hair drier', 'toothbrush' ]
names[32]

In [ ]:
import pandas as pd 
import numpy as np
box1 = [100,100,100,100]
box2 = [100,100,100,100]
state = [0,0,0,0]
df = pd.read_csv("D:/User/Bot_C/Res/scripts/control.csv")
if df.empty:
    print('DataFrame is empty!')
data_frame = []
# # data_frame.append(box1)
data_frame.append(box2)
data_frame.append(state)
data_frame = np.reshape(data_frame, (1,8))

# # df2 = pd.DataFrame(data_frame, columns=['x1', 'x2', 'x3', 'x4',
# #                                         'y1', 'y2', 'y3', 'y4',
# #                                         'q1', 'q2', 'q3', 'q4'])
df2 = pd.DataFrame(data_frame, columns=['c11', 'c12', 'c21', 'c22', 'q1', 'q2', 'q3', 'q4'])

# df2 = pd.concat([df,df2])
df2.to_csv("D:/User/Bot_C/Res/scripts/control.csv", index=False)

df2


In [1]:
import torch.nn as nn
import tqdm
import torch
class ANN(nn.Module):
    def __init__(self):
        super().__init__()

        # self.relu1 = nn.ReLU(inplace=True)
        self.liner1 = nn.Linear(8,128)
        self.relu = nn.ReLU()
        self.liner2 = nn.Linear(128,8)
        self.liner3 = nn.Linear(8,4)

        # self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.relu(self.liner1(x))
        out = self.relu(self.liner2(out))
        out = self.relu(self.liner3(out))

        return out

In [ ]:
model = ANN()
tensor = torch.rand([1, 8])
model(torch.tensor([X[0]], dtype=torch.float32))
# 

# tensor


In [3]:
def study(model, X, y, optimizer, losses, device):
    model.train()
    train_loss = 0
    train_correct = 0
    # counter = 0
    # print ("@@@ Start train model @@@")
    for i, data in enumerate(X):
        # counter += 1
        image, label = torch.tensor([data],dtype=torch.float32), torch.tensor([y[i]],dtype=torch.float32)
        image = image.to(device)
        label =label.to(device)
        optimizer.zero_grad()
        label_pre = model(image)
        # print(label_pre.shape)
        loss = losses(label_pre, label)
        train_loss += loss.item()
        # print(train_loss)
        _, preds = torch.max(label_pre.data, 1)
        train_correct += (preds == label).sum().item()
        loss.backward()
        optimizer.step()
        # print(preds)
    epoch_loss = train_loss / len(X)
    epoch_acc = train_correct / len(X)
    return epoch_loss, epoch_acc

In [5]:
from sklearn import svm
import pandas as pd
import numpy as np
import torch.optim as optim

# df = pd.read_csv("D:/User/Bot_C/Res/scripts/data_control.csv")
df = pd.read_csv("D:/User/Bot_C/Res/scripts/angles.csv")

data = np.array(df)
X = np.array(data[:,0:8]/400, dtype=np.double) 
y = np.array(data[:,8:12]/180, dtype=np.double)
# print(y)
PATH = "D:/User/Bot_C/Res/scripts/model/model_1.pt"
learning_rate = 0.01
model = ANN()
model.load_state_dict(torch.load(PATH))
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
# Loss function.
# losses = nn.CrossEntropyLoss()
losses = nn.MSELoss()

# evaluate a model using repeated k-fold cross-validation
for i in range(10000):
    train_loss, train_acc = study(model, X,y, optimizer, losses, device)
    print(train_acc, train_loss)
torch.save(model.state_dict(), PATH)


2,220 total parameters.
2,220 training parameters.
cpu
0.0 0.00042547945746187446
0.0 0.000425293947994081
0.0 0.00042521860877072035
0.0 0.0004250609923482798
0.0 0.0004249684284780376
0.0 0.0004248258221426587
0.0 0.00042469295972360143
0.0 0.00042460567134962237
0.0 0.0004244425759304117
0.0 0.0004243469562775296
0.0 0.00042419058315569595
0.0 0.0004240936465758353
0.0 0.0004239392380394162
0.0 0.00042381771023010427
0.0 0.00042374566298473065
0.0 0.0004234898075134277
0.0 0.00042337177724925505
0.0 0.000423296505405361
0.0 0.00042313513931987193
0.0 0.0004230393567179907
0.0 0.0004229069234736845
0.0 0.00042280956774059424
0.0 0.0004226766647059802
0.0 0.00042254967111532584
0.0 0.00042249573311785413
0.0 0.0004223384104024232
0.0 0.00042223439774882624
0.0 0.00042218011824249894
0.0 0.00042203184730938185
0.0 0.00042194528490432156
0.0 0.00042180925559482323
0.0 0.0004217005154005762
0.0 0.0004216281867669357
0.0 0.0004214920666189853
0.0 0.0004214247954505353
0.0 0.00042129168601

tensor([[0.5056, 0.3222, 0.6444, 0.4944]])

In [ ]:
torch.save(model.state_dict(), PATH)

In [ ]:
df_test = pd.read_csv("D:/User/Bot_C/Res/scripts/data_control.csv")
data = np.array(df_test)
X_t = np.array(data[:,0:8], dtype=np.double) 
y_t = np.array(data[:,8:12], dtype=np.double)
    

In [ ]:
from cacular import *
data_create = []
for idx, dta in enumerate(X_t):
    box_1, box_2 = dta[:4], dta[4:]
    data_frame = np.hstack(
                        [center_box(box_1), center_box(box_2), y_t[idx]])
    data_create.append(data_frame)
df = pd.read_csv("D:/User/Bot_C/Res/scripts/control.csv")
df2 = pd.DataFrame(data_create, columns=['c11', 'c12', 'c21', 'c22', 'q1', 'q2', 'q3', 'q4'])
df2 = pd.concat([df, df2])
df2.to_csv(
    "D:/User/Bot_C/Res/scripts/control.csv", index=False)
X_t
data_create

In [6]:
the_model = ANN()
the_model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [8]:
output = the_model(torch.tensor([X[:10]],dtype=torch.float32))*180
ou = np.array(output.detach().numpy()[0],dtype=np.int8)
# len(str(ou[0]))
ou

array([[ 96,  50,  99,  22],
       [108,  67,  83,  23],
       [108,  60,  79,  21],
       [ 97,  54, 104,  21],
       [104,  70,  95,  23],
       [ 96,  56, 105,  23],
       [117,  73,  72,  23],
       [ 89,  78, 108,  23],
       [ 91,  65, 102,  21],
       [104,  75, 100,  23]], dtype=int8)

In [ ]:
import torch
  
# import numpy module
import numpy
  
# create one dimensional tensor with
# float type elements
b = torch.tensor([10.12, 20.56, 30.00, 40.3, 50.4])
  
print(b)
  
# convert this into numpy array using
# numpy() method
b = b.numpy()
  
# display
b

In [ ]:
import numpy as np
np.array([-100, 100, 10,0])<np.array([0,0,0,0])